# 6 Indexed Queries in PyTables

> Objectives:
>
> * Learn how to index columns in tables for accelerating queries
> * Experiment with different indexes and/or compression
> * Discover some limitations of indexed queries.

In [4]:
import os
import numpy as np
import pandas as pd
import tables

Indexing is a general technique for adding data structures that can accelerate queries.  Let's see how PyTables makes use of this.

### Denormalized case

We'll be using the same datasets again, but we'll copy them, to add indexes: 

In [5]:
# continue from the previous notebook
data_dir = 'queries'
h5denorm = "compression/blosc-zstd-5-shuffle-denorm.h5"
h5norm = "compression/blosc-zstd-5-shuffle.h5"

In [6]:
## Copy the original PyTables table into another file
import shutil
h5idx = os.path.join(data_dir, "movielens-denorm-indexed.h5")
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5denorm, h5idx)

'queries\\movielens-denorm-indexed.h5'

In [7]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")

In [8]:
# Create an index for the 'title' column
h5lens = h5i.root.lens
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5lens.cols.title.create_csindex(filters=blosc_filter)

Wall time: 3.24 s


1000209

In [9]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

Wall time: 11.1 ms


In [10]:
ratings

[0, 4, 15, 28, 18, 3]

Ok, so this time is 100x less than without using indexing.  What if we index the `rating` column too?

In [11]:
# Create an index for the rating column
%time h5lens.cols.rating.create_csindex(filters=blosc_filter)

Wall time: 770 ms


1000209

In [12]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

Wall time: 6.04 ms


Ok, so although small, this represents another improvement in performance.

In [13]:
ratings

[0, 4, 15, 28, 18, 3]

In [14]:
h5i.close()

### Normalized case

In [15]:
## Copy the original PyTables table into another file
import shutil
h5idx = os.path.join(data_dir, "movielens-norm-indexed.h5")
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5norm, h5idx)

'queries\\movielens-norm-indexed.h5'

In [16]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

In [17]:
# Create an index for the rating column
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5ratings.cols.rating.create_csindex(filters=blosc_filter)

Wall time: 577 ms


1000209

In [18]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

Wall time: 436 ms


Hmm, in this case indexing the rating column has not served to accelerate the query (at first sight at least).

In [19]:
ratings

[0, 4, 15, 28, 18, 3]

In [20]:
# Create an index for the movie_id column
%time h5ratings.cols.movie_id.create_csindex(filters=blosc_filter)

Wall time: 700 ms


1000209

In [21]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

Wall time: 46 ms


This time we see a better acceleration in the query, but cannot compete with the query speed for the denormalized case (which is ~10x faster).

In [22]:
ratings

[0, 4, 15, 28, 18, 3]

In [23]:
h5i.close()

In [24]:
!ls -lh {data_dir}

total 20M
-rw-r--r-- 1 tomkooij 197613 9.9M Jun 26 10:27 movielens-denorm-indexed.h5
-rw-r--r-- 1 tomkooij 197613 9.6M Jun 26 10:27 movielens-norm-indexed.h5


## Exercise

We have not created an index for the title for the normalized case.  Create such an index and determine if there is a noticeable speed-up or not.  Explain why you think that is the case.  Note: the times for a cold query can be **significatively** different from a hot query.

In [33]:
## Copy the original PyTables table into another file
import shutil
h5idx2 = "movielens-norm-indexed2.h5"
if os.path.exists(h5idx2):
    os.unlink(h5idx2)
shutil.copyfile(h5idx, h5idx2)

'movielens-norm-indexed2.h5'

In [34]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx2, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

In [35]:
#
#
# Solution starts here
#
#

In [36]:
# Create an index for the movie_id column
%time h5movies.cols.title.create_csindex(filters=blosc_filter)

Wall time: 16 ms


3883

In [38]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

Wall time: 43.1 ms


In [39]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

Wall time: 40.5 ms


In [40]:
ratings

[0, 4, 15, 28, 18, 3]

In [41]:
h5i.close()

So the first time that the query is done after the cache is built (cold query), the time has been reduced a bit but not too much.  For subsequent queries (hot queries), the times are better, but not reaching the denormalized table either.

# Exercise

Query size vs speed (indexed queries vs non-indexed queries)

We create a (large) file containing some `(key, value)` pairs. The `value` is an `int64`. The `key` is a random 10-byte string, to simulate actual data, with normal compression.

Investigate the query speed vs query result size. **Create the datafile first, the assignment is below**

In [42]:
N = 20  # append 20 blocks of 1M rows

In [43]:
# adapted from: https://stackoverflow.com/questions/20769818/

import random
import string

class KeyValue(tables.IsDescription):
    key = tables.StringCol(itemsize=10, dflt=" ", pos=0)  
    value = tables.Int64Col(dflt=0, pos=1)

fn = os.path.join(data_dir, "keyvalue.h5")

with tables.open_file(fn, "w") as f:    
    filters = tables.Filters(complevel=5, complib='blosc')
    kv = f.create_table("/", "keyvalues", KeyValue, filters=filters)

    for j in range(1, N+1):
        values = []
        print('block: ', j)
        for _ in range(100000):
            key = "".join(random.sample(string.ascii_uppercase, 10))  # slow!
            value = random.randint(0, 1000000)
            values.append((key, value))
        kv.append(values)

block:  1
block:  2
block:  3
block:  4
block:  5
block:  6
block:  7
block:  8
block:  9
block:  10
block:  11
block:  12
block:  13
block:  14
block:  15
block:  16
block:  17
block:  18
block:  19
block:  20


In [44]:
!ptdump -v -R10 {fn}

/ (RootGroup) ''
/keyvalues (Table(2000000,), shuffle, blosc(5)) ''
  description := {
  "key": StringCol(itemsize=10, shape=(), dflt=b' ', pos=0),
  "value": Int64Col(shape=(), dflt=0, pos=1)}
  byteorder := 'little'
  chunkshape := (3640,)
  Data dump:
[0] (b'EIMUAQBPSZ', 671967)
[1] (b'OIAWTMXSEN', 347907)
[2] (b'EKWDBAFQSO', 718439)
[3] (b'ZCEUWVIXYT', 180163)
[4] (b'GHUCJAYXOB', 615020)
[5] (b'MGQAXHJDVI', 421151)
[6] (b'GPFLRWTNVA', 148149)
[7] (b'TDKXBYULVR', 894572)
[8] (b'QAORSBVIDC', 197511)
[9] (b'JWXGUYTNZA', 276217)



Query the `value` column and compare different query (result) sizes:
Compare indexed queries with unindexed queries.
*Optional: compare different compression levels and codecs* 


For example: `'(value > 100000) & (value <1000010)'`


In [ ]:
#
#
# Results start here
#
#


In [49]:
max_values = [10, 50, 100, 1000, 10000]
X = 100000

def get_query(max_value):
    return '(value > %s) & (value <%s)' % (X, X+max_value)


with tables.open_file(fn, "a") as f:
    kv = f.root.keyvalues
    #kv = f.root.sorted
    
    kv.cols.value.remove_index()
    
    
    for max_value in max_values:
        query = get_query(max_value)
        print('max_value=%d : len=%d' % (max_value, len(kv.read_where(query))))
    
    print('\nread entire table:')
    %timeit kv.read()
    
    print('\nwithout index:')
    for max_value in max_values:
        query = get_query(max_value)
        %timeit sum(1 for x in kv.where(query))

    blosc_filter = tables.Filters(complevel=9, complib="blosc")
    print('\nindexing...')
    %time kv.cols.value.create_csindex()

    print('\nwith index')
    for max_value in max_values:
        query = get_query(max_value)
        %timeit sum(1 for x in kv.where(query))

max_value=10 : len=18
max_value=50 : len=83
max_value=100 : len=176
max_value=1000 : len=1993
max_value=10000 : len=19874

read entire table:
46.7 ms ± 4.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

without index:
45.9 ms ± 7.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
51.2 ms ± 6.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
52.9 ms ± 5.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
47.3 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
49 ms ± 798 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

indexing...
Wall time: 3.25 s

with index
173 µs ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
3.42 ms ± 348 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.26 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
59.9 ms ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
66.4 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Exercise (Optional)

Indexing queries with large result sets is difficult. `pytables` is not optimised for such queries. In general results are comparable to unindexed queries (reading the entire table).

For extreme performance, try an indexed query on a sorted table.

In [50]:
%%time
with tables.open_file(fn, 'a') as f:
    table = f.root.keyvalues[:]
    table.sort(order='value')
    f.create_table('/', 'sorted', obj=table)

Wall time: 4.72 s


In [51]:
!ptdump -v {fn}

/ (RootGroup) ''
/keyvalues (Table(2000000,), shuffle, blosc(5)) ''
  description := {
  "key": StringCol(itemsize=10, shape=(), dflt=b' ', pos=0),
  "value": Int64Col(shape=(), dflt=0, pos=1)}
  byteorder := 'little'
  chunkshape := (3640,)
  autoindex := True
  colindexes := {
    "value": Index(9, full, shuffle, zlib(1)).is_csi=True}
/sorted (Table(2000000,)) ''
  description := {
  "key": StringCol(itemsize=10, shape=(), dflt=b'', pos=0),
  "value": Int64Col(shape=(), dflt=0, pos=1)}
  byteorder := 'little'
  chunkshape := (3640,)


In [52]:
max_values = [10, 50, 100, 1000, 10000]
X = 100000

def get_query(max_value):
    return '(value > %s) & (value <%s)' % (X, X+max_value)


with tables.open_file(fn, "a") as f:
    #kv = f.root.keyvalues
    kv = f.root.sorted
    
    kv.cols.value.remove_index()

    for max_value in max_values:
        query = get_query(max_value)
        print('max_value=%d : len=%d' % (max_value, len(kv.read_where(query))))
    
    print('\nread entire table:')
    %timeit kv.read()

    print('\nwithout index:')
    for max_value in max_values:
        query = get_query(max_value)
        %timeit sum(1 for x in kv.where(query))

    blosc_filter = tables.Filters(complevel=9, complib="blosc")
    print('\nindexing...')
    %time kv.cols.value.create_csindex()

    print('\nwith index')
    for max_value in max_values:
        query = get_query(max_value)
        %timeit sum(1 for x in kv.where(query))

max_value=10 : len=18
max_value=50 : len=83
max_value=100 : len=176
max_value=1000 : len=1993
max_value=10000 : len=19874

read entire table:
42.2 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

without index:
39.2 ms ± 4.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.1 ms ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.6 ms ± 711 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.6 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
36 ms ± 336 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

indexing...
Wall time: 2.5 s

with index
143 µs ± 380 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
212 µs ± 7.28 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
302 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.06 ms ± 6.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.74 ms ± 5.19 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
